In [1]:
# !pip3 install pygmo

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import src.mo as mo

### Coisas a fazer
#### Comparar métodos

* ~~calcular hipervolume~~
* colocar duas curvas de pareto no mesmo gráfico
* ~~contar pontos não dominados~~
* criar curva de pareto a partir de pontos
* ~~cores diferentes para otimo/factivel~~

In [2]:
from src.util_experiments import clean_sol_csv_file

filepath = "../project/resultados/exp_20220927/A-n50-m4-Q80-v4-s100-d1_1_1_1_sol_20220927135621.csv"
df = clean_sol_csv_file(filepath)

In [3]:
from src.util_experiments import read_out_file, read_pos_processed_csv

nsga_pp_path = "../project/resultados/exp_20221005/n50/A-n50-m4-Q160-v4-s100-d1_1_1_1-20221005194618.csv"
df_nd = read_pos_processed_csv(nsga_pp_path)
df_nd.head()

,dist,demand
0,0.00000,0.0
1,7.07107,15.0
2,8.54400,19.0
3,15.25220,45.0
4,16.60630,48.0


In [ ]:
def plot_status_nsga(model_path=None, nsga_path=None, nsga_pp_path=None, title='', figsize=(9, 9)):
    if model_path == None and nsga_path == None and nsga_pp_path == None:
        raise Exception("Both paths are None")
        
    fig, ax = plt.subplots(figsize=figsize)
    color_list = mcolors.TABLEAU_COLORS
    
    plt.xlabel('Distância')
    plt.ylabel('- Demanda')
    plt.title(title)
    
    if model_path != None:
        df_model = clean_sol_csv_file(model_path)

        all_status = list(set(df_model.status))
        color_sequence = {i:color for i, color in enumerate(color_list)}

        color_map = {el:color_sequence[i] for i, el in enumerate(all_status)}

        for status in all_status:
            df_ = df_model[df_model.status == status]

            x = df_.dist
            y = -df_.demand
            colors = [color_map[status] for status in df_.status]

            ax.scatter(x, y, c=colors, alpha=0.5)
    
    if nsga_path != None:
        df_, params = read_out_file(nsga_path)
        
        ax.scatter(df_.dist, -df_.demand, c='tab:gray', marker='v', alpha=0.5)
        all_status.append('NSGAII')
    
    if nsga_pp_path != None:
        df_pp_nsga = read_pos_processed_csv(nsga_pp_path)
        
        ax.scatter(df_pp_nsga.dist, -df_pp_nsga.demand, c='tab:red', marker='*', alpha=0.5)
        all_status.append('NSGAII_pp')
    
    ax.legend(all_status)
    
    return fig, ax

# model_path = "../project/resultados/exp_20220927/A-n50-m4-Q80-v4-s100-d1_1_1_1_sol_20220927135621.csv"
model_path = "../project/resultados/exp_20220927/A-n50-m4-Q80-v10-s100-d3_3_2_2_sol_20220927125957.csv"

nsga_path = "../project/resultados/exp_20221005/n50/A-n50-m4-Q160-v4-s100-d1_1_1_1-20221005194618.out"
nsga_pp_path = "../project/resultados/exp_20221005/n50/A-n50-m4-Q160-v4-s100-d1_1_1_1-20221005194618.csv"


fig, ax = plot_status_nsga(model_path, nsga_path, nsga_pp_path, figsize=(5, 5))
# fig, ax = plot_status_nsga(model_path, nsga_path, None, figsize=(5, 5))

In [ ]:
import glob
model_path = "../project/resultados/exp_20220927/A-n50-m4-Q80-v10-s100-d3_3_2_2_sol_20220927125957.csv"

nsga_path = "../project/resultados/exp_20221005/n50/A-n50-m4-Q160-v4-s100-d1_1_1_1-20221005194618.out"
nsga_pp_path = "../project/resultados/exp_20221005/n50/A-n50-m4-Q160-v4-s100-d1_1_1_1-20221005194618.csv"


# fig, ax = plot_status_nsga(model_path, nsga_path, nsga_pp_path, figsize=(5, 5))

for filepath in glob.glob("../project/resultados/exp_20221005/n50/A-n50-m4-Q160-v4-s100-d1_1_1_1*.out"):
    pp_path = filepath.replace('out', 'csv')
    fig, ax = plot_status_nsga(model_path, filepath, pp_path, figsize=(5, 5))
    

In [ ]:
fig, ax = plot_status_nsga(model_path, nsga_path, None, figsize=(5, 5))

In [ ]:
%ls ../project/resultados/exp_20220927/A-n50-m4-Q80-v4-s100*

In [ ]:
# for i in range(3):
#     plot_scatter(df, figsize=(4,4))
import os
import glob
import pygmo as pg

folderpath = "../project/resultados/exp_20220927"

inst_name_from_filepath = lambda filepath: os.path.basename(filepath).split('_sol')[0]

for filepath in glob.glob(f"{folderpath}/*_sol_*.csv"):
    if os.path.isfile(filepath):
        instance = inst_name_from_filepath(filepath)
        
        df = clean_sol_csv_file(filepath)
#         plot_scatter(df, figsize=(4,4), title=instance)
        
        plt_scatter_dom_not_dom(df, figsize=(4,4), title=instance)
        
#         pop = df[['dist', 'demand']].values*[1, -1]
        
#         ref_point = mo.ref_point(pop)
#         print(f"{instance}: {ref_point}")
#         print(mo.hypervolume(pop, ref_point))
        
#         hv = pg.hypervolume(pop)
        
#         print('*'*80)
#         contri = hv.contributions(ref_point)
#         for c in contri:
#             if c > 0:
#                 print(c)
        
#         print(f"{instance}: {mo.hypervolume(pop, ref_point)/ref_point[0]/ref_point[1]}")
        

In [ ]:
def plt_scatter_dom_not_dom(pop, title="", figsize=(9, 9)):
    no_dom_idx, dom_idx = mo.no_dominated(pop)
    
    fig, ax = plt.subplots(figsize=figsize)
    
    plt.xlabel('Distância')
    plt.ylabel('- Demanda')
    plt.title(title)
    
    plt_idx = lambda idx, color, alpha: ax.scatter(pop[idx, 0], pop[idx, 1], c=color, alpha=alpha)
    
    plt_idx(no_dom_idx, 'tab:green', 0.7)
    plt_idx(dom_idx, 'tab:gray', 0.5)
    
    ax.legend(['Not dominated', 'Dominated'])
    return (fig, ax)

df_model = clean_sol_csv_file(model_path)
pop = df_model[['dist', 'demand']].values*[1, -1]
fig, ax = plt_scatter_dom_not_dom(pop, figsize=(5, 5))

df_, params = read_out_file(nsga_path)
pop = df_[['dist', 'demand']].values*[1, -1]

fig, ax = plt_scatter_dom_not_dom(pop, figsize=(5, 5))